# Otros métodos para sistemas de ecuaciones
En este cuaderno veremos algunos ejemplos de métodos para resolución de sistemas de ecuaciones lineales.

En este cuaderno veremos los métodos de Thomas, gauss-seidel y Jacobi

Comenzamos importando numpy y scypi, los cuales nos ayudan a trabajar con matrices y vectores


In [10]:
import numpy as np
import scipy

## Matrices tridiagonales

Recordemos que podemos definir una matriz tridiagonal solamente estableciendo sus 3 vectores diagonales.
Con lo cual creamos estos de manera aleatoria. Se agrega uno para asegurar que ninguno de estos elementos sea cero y concatenamos un cero al final de a y c para que las longitudes de estos vectores coincidan


In [11]:
a = np.concatenate((np.array([0]), np.random.randint(10, size=4) +1) )
b = np.random.randint(10, size=5) +1 
c = np.concatenate((np.random.randint(10, size=4) +1 , np.array([0])))
print(a,b,c)

[ 0  2 10  4  5] [4 4 7 6 9] [ 7  3  8 10  0]


Definimos un par de funciones que nos pueden ayudar a transformar de la representación en 3 vectores a la representación matricial.

In [12]:
def trivector2tridiagonal(a,b,c):
  matriz = np.diag(a[1:],-1) +  np.diag(b,0) +  np.diag(c[:-1],1)
  return matriz

def tridiagonal2trivector(matriz):
  a = np.concatenate((np.array([0]),np.diag(matriz,-1)))
  b = np.diag(matriz,0)
  c = np.concatenate((np.diag(matriz,1), np.array([0])))
  return a,b,c


Podemos ver cómo pasar de la representación en 3 vectores a la representación matricial.

In [13]:
m = trivector2tridiagonal(a,b,c)
m

array([[ 4,  7,  0,  0,  0],
       [ 2,  4,  3,  0,  0],
       [ 0, 10,  7,  8,  0],
       [ 0,  0,  4,  6, 10],
       [ 0,  0,  0,  5,  9]])

Tenemos la transformación inversa donde con una matriz obtenemos los 3 vectores diagonales

In [14]:
tridiagonal2trivector(m)

(array([ 0,  2, 10,  4,  5]),
 array([4, 4, 7, 6, 9]),
 array([ 7,  3,  8, 10,  0]))

Definimos un sistema de ecuaciones aleatorio, dónde establecemos el valor de x al azar. Con esto podemos calcular el vector independiente de nuestro sistema de ecuaciones.

Imaginemos que no conocemos el vector x y que lo queremos encontrar.

In [15]:
x = np.random.randint(10, size=(5,1)) - 5

vector_independiente = np.matmul(m,x)
print(x,vector_independiente)

[[ 0]
 [-5]
 [ 0]
 [ 2]
 [-4]] [[-35]
 [-20]
 [-34]
 [-28]
 [-26]]


### Método de Thomas

Creamos la función método tomas dónde seguiremos la metodología de Thomas 

In [16]:
def metodo_thomas(matriz_tridiagonal,vector_independiente):
  a, b, c = tridiagonal2trivector(matriz_tridiagonal)
  n = len(a)
  d = vector_independiente.flatten()
  a_ = np.zeros_like(a)
  b_ = b.astype(float)
  c_ = c.astype(float)
  d_ = d.astype(float)

  for i in range(len(b)-1):
    factor = a[i+1]/b_[i]
    b_[i+1] = b[i+1] - factor*c_[i]
    d_[i+1] = d[i+1] - factor*d_[i]

  x = np.zeros_like(d,float)
  x[n-1] = d_[n-1]/b_[n-1]
  for i in range(n-2,-1,-1):
    x[i] = (d_[i]-(c_[i]*x[i+1]))/b_[i]

  return x


Aplicamos este método a el sistema de ecuaciones creado

In [17]:
metodo_thomas(m,vector_independiente)

array([ 0., -5., -0.,  2., -4.])

## Descomposición LU
Creamos un nuevo sistema de ecuaciones


In [18]:
A = np.random.randint(20, size=(5,5)) - 10
x = np.random.randint(20, size=(5,1)) - 10
b = np.matmul(A,x)

print(A,x,b)

[[-7  0  3  4  8]
 [-5 -3 -4  1 -6]
 [ 9  3 -1  4  4]
 [ 8  9  9  2  1]
 [ 1 -3 -7 -3 -7]] [[ 1]
 [ 8]
 [ 9]
 [-1]
 [ 0]] [[ 16]
 [-66]
 [ 20]
 [159]
 [-83]]


Utilizamos scipy para generar la descomposición LU 

In [21]:
L,U = scipy.linalg.lu(A,True)
print(L,U)
#P,L,U = scipy.linalg.lu(A,False)
#print(P,L,U)

[[-0.77777778  0.36842105  0.57446809 -0.96587031  1.        ]
 [-0.55555556 -0.21052632  1.          0.          0.        ]
 [ 1.          0.          0.          0.          0.        ]
 [ 0.88888889  1.          0.          0.          0.        ]
 [ 0.11111111 -0.52631579  0.68085106  1.          0.        ]] [[ 9.          3.         -1.          4.          4.        ]
 [ 0.          6.33333333  9.88888889 -1.55555556 -2.55555556]
 [ 0.          0.         -2.47368421  2.89473684 -4.31578947]
 [ 0.          0.          0.         -6.23404255 -5.85106383]
 [ 0.          0.          0.          0.          8.88054608]]


Note que L no es una matriz triangular inferior sino que está permutada para mejorar el desempeño y evitar errores numéricos.

Resolvemos el sistema intermedio con la matriz inferior y el vector independiente y guardamos el resultado


In [22]:
c = np.linalg.solve(L,b)
c

array([[ 2.00000000e+01],
       [ 1.41222222e+02],
       [-2.51578947e+01],
       [ 6.23404255e+00],
       [-8.88178420e-15]])

El resultado será el vector independiente para resolver respecto a la matriz triangular superior

In [23]:
y = np.linalg.solve(U,c)
y

array([[ 1.00000000e+00],
       [ 8.00000000e+00],
       [ 9.00000000e+00],
       [-1.00000000e+00],
       [-1.00013942e-15]])

## Métodos Iterativos


Creamos un sistema de ecuaciones aleatorio el cual vamos a incrementar su diagonal para asegurar la convergencia de los métodos.

In [24]:
A = np.random.randint(20, size=(5,5)) - 10
A += np.diag(np.diag(A))*5 #Incrementamos la diagonal para facilitar los cálculos
x = np.random.randint(20, size=(5,1)) - 10
b = np.matmul(A,x)
print(A,x,b)

[[-42  -1  -6  -2  -9]
 [  2 -36  -8  -1   2]
 [-10  -5  48   6   3]
 [ -7  -6 -10 -42  -8]
 [  7  -7   2  -9  12]] [[-7]
 [ 5]
 [-6]
 [ 7]
 [ 5]] [[ 266]
 [-143]
 [-186]
 [-255]
 [ -99]]



### Método de Jacobi

definimos la función la cual sigue la metodología de Jacobi realizando multiplicación de matrices


In [25]:
def jacobi(A,b,L=25,x=None):
  n = len(A[0])
  if x is None:
      x = np.zeros(n).reshape((n,1))

  D = np.diag(A).reshape((n,1))
  B = -A/D + np.eye(n)
  c = b/D
  for k in range(L):
    print(k,np.transpose(x))
    x = np.matmul(B,x) + c
    
  return x

Ejecutamos el método de Jacobi dónde nos da la aproximación del vector de incógnitas a cada paso.

In [26]:
jacobi(A,b)

0 [[0. 0. 0. 0. 0.]]
1 [[-6.33333333  3.97222222 -3.875       6.07142857 -8.25      ]]
2 [[-4.39559713  3.85449735 -5.02397487  9.05357143  2.96097884]]
3 [[-6.7730143   4.75747197 -5.70599687  6.88557414  4.19006283]]
4 [[-6.85721916  4.90545827 -5.91305034  7.08108891  4.59129707]]
5 [[-6.92645256  4.96365193 -5.96469427  7.04685503  4.90788691]]
6 [[-6.98667092  4.98982133 -5.98856368  7.0020739   4.96515128]]
7 [[-6.99402258  4.99620545 -5.99636458  7.0031475   4.98593652]]
8 [[-6.99756528  4.99865547 -5.99866444  7.00135902  4.99605441]]
9 [[-6.99937801  4.99958152 -5.9995561   7.00021984  4.99859211]]
10 [[-6.99976223  4.99985159 -5.9998535   7.0001186   4.99948396]]
11 [[-6.99991246  4.99994869 -5.9999485   7.00004499  4.99983926]]
12 [[-6.99997383  4.99998324 -5.99998268  7.0000111   4.99994416]]
13 [[-6.99999064  4.9999942  -5.99999419  7.00000455  4.99998039]]
14 [[-6.99999671  4.99999801 -5.999998    7.00000162  4.99999359]]
15 [[-6.99999894  4.99999934 -5.99999932  7.000000

array([[-7.],
       [ 5.],
       [-6.],
       [ 7.],
       [ 5.]])

### Método de Gauss-Seidel

Ahora definimos la función para el método de gauss seidel la cual comparte en gran medida con la función de Jacobi.


In [27]:
def gauss_seidel(A,b,L=25,x=None):
  n = len(A[0])
  if x is None:
      x = np.zeros(n)

  D = np.diag(A).reshape((n,1))
  B = -A/D + np.eye(n)
  c = b/D
  for k in range(L):
    for i in range(n):
      x[i] = np.matmul(B[i],x) + c[i]
    print(k,np.transpose(x))
    
  return x

De la misma manera la utilizamos para resolver el sistema de ecuaciones, vemos que este converge de manera mas rápida. 

In [28]:
gauss_seidel(A,b)

0 [-6.33333333  3.62037037 -4.81732253  7.75676991  4.17679169]
1 [-6.99574048  4.67066424 -6.07656413  7.22136918  4.98419033]
2 [-6.98837454  5.01063265 -6.02315351  7.00506758  5.00708046]
3 [-6.99870407  5.00546981 -6.00023622  6.99771019  5.00075678]
4 [-7.00014962  5.00014983 -5.99977664  6.9998062   4.9999921 ]
5 [-7.00002456  4.99995394 -5.99998519  7.00000865  4.99999148]
6 [-6.9999996   4.99999602 -6.00000088  7.00000234  4.99999935]
7 [-6.99999975  5.00000011 -6.00000019  7.00000011  5.00000003]
8 [-6.99999999  5.00000004 -6.00000001  6.99999999  5.00000001]
9 [-7.  5. -6.  7.  5.]
10 [-7.  5. -6.  7.  5.]
11 [-7.  5. -6.  7.  5.]
12 [-7.  5. -6.  7.  5.]
13 [-7.  5. -6.  7.  5.]
14 [-7.  5. -6.  7.  5.]
15 [-7.  5. -6.  7.  5.]
16 [-7.  5. -6.  7.  5.]
17 [-7.  5. -6.  7.  5.]
18 [-7.  5. -6.  7.  5.]
19 [-7.  5. -6.  7.  5.]
20 [-7.  5. -6.  7.  5.]
21 [-7.  5. -6.  7.  5.]
22 [-7.  5. -6.  7.  5.]
23 [-7.  5. -6.  7.  5.]
24 [-7.  5. -6.  7.  5.]


array([-7.,  5., -6.,  7.,  5.])